# Core Imports and Setup

In [1]:
import os
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger("openff.toolkit").setLevel(logging.ERROR)

from openff import toolkit, evaluator

In [2]:
import logging
from collections import defaultdict
from enum import Enum
from os import path

import numpy as np
from openff.units import unit
from openmm import app

from openff.evaluator.forcefield import SmirnoffForceFieldSource
from openff.evaluator.attributes import UNDEFINED
from openff.evaluator.substances import Component, ExactAmount, MoleFraction, Substance
from openff.evaluator.utils import packmol
from openff.evaluator.workflow import Protocol, workflow_protocol
from openff.evaluator.workflow.attributes import InputAttribute, OutputAttribute
from openff.evaluator.protocols import coordinates, forcefield
from openff.evaluator.workflow import utils
from openff.evaluator.workflow import Workflow

# Step 1: generating force field

In [3]:
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.evaluator.forcefield import SmirnoffForceFieldSource

# Evaluator wants to work with a JSON file for the force field
force_field = ForceField("openff-1.0.0.offxml")
with open("force-field.json", "w") as file:
    file.write(SmirnoffForceFieldSource.from_object(force_field).json())

# Option 1: generating default metadata

This pathway generates default metadata from an already-defined physical property.

In [4]:
from openff.units import unit
from openff.evaluator import properties
from openff.evaluator.datasets.thermoml import thermoml_property
from openff.evaluator.datasets import PhysicalProperty, PropertyPhase
from openff.evaluator.properties import Density, EnthalpyOfMixing
from openff.evaluator.client import RequestOptions
from openff.evaluator.thermodynamics import ThermodynamicState

# create substance from multiple components
substance = Substance()
substance.add_component(Component(smiles='O'), MoleFraction(0.2))
substance.add_component(Component(smiles='CO'), MoleFraction(0.8))

# create enthalpy of mixing property target
thermodynamic_state = ThermodynamicState(
    temperature=298 * unit.kelvin,
    pressure=1 * unit.atmosphere
)
physical_property = EnthalpyOfMixing(
    thermodynamic_state=thermodynamic_state,
    phase=PropertyPhase.Liquid,
    substance=substance,
    value=10.0 * unit.kilojoules_per_mole,
    uncertainty=1.0 * unit.kilojoules_per_mole,
)

# create default metadata
metadata = Workflow.generate_default_metadata(
    physical_property,
    "force-field.json"
)

# Option 2: manually generating metadata without pre-defined values

In [5]:
components = [
    Substance.from_components("CO"),
    Substance.from_components("O"),
]

metadata = {
    "substance": substance,  # from cell defined above
    "components": components,
    "thermodynamic_state": thermodynamic_state,  # from cell defined above
    "force_field_path": "force-field.json",
    "parameter_gradient_keys": []
}

# Creating workflow from schema and executing

In [6]:
h_mix_schema = EnthalpyOfMixing.default_simulation_schema()
workflow_schema = h_mix_schema.workflow_schema

# replace default system building with SMIRNOFF type for a smirnoff force field
# (can also have tleap, ligpargen)
workflow_schema.replace_protocol_types(
    {"BaseBuildSystem": "BuildSmirnoffSystem"}
)

# quickly hacking and re-setting steps to finish fast
# don't do this in production!
for schema in workflow_schema.protocol_schemas:
    # equilibration_simulation is a Protocol
    if "simulation" in schema.id:
        schema.inputs[".steps_per_iteration"] = 10
        schema.inputs[".output_frequency"] = 10
    # conditional_group is a group of protocols
    if "conditional" in schema.id:
        for protocol_name, protocol in schema.protocol_schemas.items():
            if "simulation" in protocol_name:
                protocol.inputs[".steps_per_iteration"] = 10
                protocol.inputs[".output_frequency"] = 10

workflow = Workflow.from_schema(workflow_schema, metadata=metadata)

In [7]:
workflow_schema.protocol_schemas[-8].inputs

{'.allow_merging': True,
 '.steps_per_iteration': 10,
 '.total_number_of_iterations': 1,
 '.output_frequency': 10,
 '.checkpoint_frequency': 10,
 '.timestep': 2.0 <Unit('femtosecond')>,
 '.thermodynamic_state': <ProtocolPath full_path=global.thermodynamic_state>,
 '.ensemble': <Ensemble.NPT: 'NPT'>,
 '.thermostat_friction': 1.0 <Unit('1 / picosecond')>,
 '.input_coordinate_file': <ProtocolPath full_path=energy_minimisation_mixture.output_coordinate_file>,
 '.parameterized_system': <ProtocolPath full_path=assign_parameters_mixture.parameterized_system>,
 '.enable_pbc': True,
 '.allow_gpu_platforms': True,
 '.high_precision': False,
 '.gradient_parameters': []}

# Executing workflow

In [8]:
result = workflow.execute()

In [9]:
result.value

<Measurement(-0.14433017504767465, 8.379058453350834e-16, kilojoule / mole)>